# Restricted Boltzmann Machines (RBMs)
Let us assume we have some random samples of fixed-length binary sequences that we wish to express in an RBM.

In [ ]:
import numpy as np
data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])

### Constructing the model
Our RBM has 6 visible, and say 2 hidden units.

In [ ]:
num_visible = 6 # = data.shape[0]
num_hidden  = 2

The RBM is defined by a set of weights, to which we add the biases.

$E(v, h)=v^T W h + b^T h$

In [ ]:
weights = np.random.randn(num_visible, num_hidden)
weights

In [ ]:
weights = np.insert(weights, 0, 0, axis=0)
weights

In [ ]:
weights = np.insert(weights, 0, 0, axis=1)
weights

In [ ]:
weights.shape

### Training the model

In [ ]:
nb_epoch = 100
learning_rate = 0.1
training_data = np.insert(data, 0, 1, axis=1)
num_examples = training_data.shape[0]
training_data


In [ ]:
training_data.shape

To train the model, we need to compute the hidden activations, probabilities, and states.

In [ ]:
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def compute_hidden(visible):
    num_examples = visible.shape[0]
    pos_hidden_activations = np.dot(visible, weights)      
    pos_hidden_probs = logistic(pos_hidden_activations)
    pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, num_hidden + 1)
    return pos_hidden_activations, pos_hidden_probs, pos_hidden_states

test_data = training_data[:1]
a, p, hidden_states = compute_hidden(test_data)
print("input data:", test_data)
print("activation:", a)
print("probabilit:", p)
print("hidden sta:", hidden_states)

We also need to compute the visible units to compare with the visible activations, probs., and states.

In [ ]:
def compute_visible(hidden_states):
    num_examples = hidden_states.shape[0]
    neg_visible_activations = np.dot(hidden_states, weights.T)
    neg_visible_probs = logistic(neg_visible_activations)
    neg_visible_probs[:, 0] = 1 # Fix the bias unit.
    return neg_visible_probs

neg_visible_probs = compute_visible(s)
print("negative visible probabilities:", visible_probs)

In [ ]:
def daydream(neg_visible_probs):
    neg_hidden_activations = np.dot(neg_visible_probs, weights)
    neg_hidden_probs = logistic(neg_hidden_activations)
    return neg_hidden_probs

In [ ]:
for epoch in range(nb_epoch):
    pos_hidden_act, pos_hidden_prob, pos_hidden_stat = compute_hidden(training_data)
    
    neg_visible_prob = compute_visible(pos_hidden_stat)
    neg_hidden_prob = daydream(neg_visible_prob)
    
    pos_associations = np.dot(training_data.T, pos_hidden_prob)
    neg_associations = np.dot(neg_visible_prob.T, neg_hidden_prob)
    
    # Update:
    weights += learning_rate * ((pos_associations-neg_associations)/float(num_examples))
    
    # Error:
    err = np.sum( (training_data - neg_visible_prob)**2 )
    print("Epoch %i, error: %.3g" % (epoch+1, err))